## Agile Hardware Design
***
# FP Conclusion + Pattern Matching

## Prof. Scott Beamer
### sbeamer@ucsc.edu

## [CSE 293](https://classes.soe.ucsc.edu/cse293/Spring21/)

## Plan for Today

* FP conclusion: flatMap, filter, sum
* Pattern matching
* Gracefully handling Option

## Loading The Chisel Library Into a Notebook

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

## Scala `flatMap`

* Like `map`, but concatenates output of function's
  * More direct than calling `map` and then `flatten`
* Helpful for aggregating results of map when they are a collection

In [ ]:
val l = 0 until 5
Seq.fill(2)(3)
l map { i => Seq.fill(i)(i) }
// l map { i => Seq.fill(i)(i) } flatten
// l flatMap { i => Seq.fill(i)(i) }

## Applying Predicates in Scala

* A _predicate_ is a function that given a single element returns a `Boolean`
* `filter` - elements persists to output collection only if predicate returns true
* `forall` - true if and only if predicate is true for all elements
* `exists` - true if predicate is true for at least one element

In [ ]:
def isEven(x: Int): Boolean = x % 2 == 0
val l = 0 until 5
l filter isEven
l filterNot isEven
l forall isEven
l exists isEven

## Example: Prime Seive in Scala

In [ ]:
def multipleOf(a: Int)(b: Int): Boolean = (a != b) && (b % a == 0)

def removeX(l: Seq[Int], x: Int) = l filterNot multipleOf(x)

val allNums = 2 until 100

// allNums filterNot multipleOf(5)
// removeX(allNums, 5)

def seive(s: Seq[Int]): Seq[Int] = {
    if (s.isEmpty) Seq()
    else Seq(s.head) ++ seive(removeX(s.tail, s.head))
}

println(seive(allNums))

## Scala Has Common Reductions Built-in

* `sum`, `product`, `min`, `max`

In [ ]:
val l = 0 until 5
l reduce { _ + _ }
l.sum
l reduce { _ * _ }
l.product
l reduce { _ min _ }
l.min
l reduce { _ max _ }
l.max

## Example: Using FP to do Matrix Multiplication

In [ ]:
val mat = Seq.tabulate(4,4){ (i,j) => i+j }

def grabCol(m: Seq[Seq[Int]], i: Int) = m map { _(i) }

def dotP(a: Seq[Int], b: Seq[Int]) = a.zip(b).map{ case (x,y) => x * y}.sum

def matMul(a: Seq[Seq[Int]], b: Seq[Seq[Int]]) = a map {
    row => row.zipWithIndex map {
        case (element, colIndex) => dotP(row, grabCol(b, colIndex))
    }
}

matMul(mat, mat)

## Pattern Matching in Scala

* Can gracefully scale from replacing simple `if/else` and `switch` cases to more sophisticated searches
* Start block with `match` and list matches with `case`
* Can use `|` for or
* Can use `if` to specify condition
* Can use `_` for default (matched nothing above)

In [ ]:
val x = 10

x match {
    case 0 => "0"
    case 1 | 3 => "nah"
    case x if (x%2 == 0) => "even"
    case 5 => "found it!"
    case _ => "other"
}

## Can Match on Case Classes

* Can match on type as a whole, or even set fields

In [ ]:
abstract class Vehicle

case class helicopter(color: String, driver: String) extends Vehicle

case class submarine(color: String, driver: String) extends Vehicle

val movers = Seq(helicopter("grey", "Marta"), helicopter("blue", "Laura"), submarine("yellow", "Paul"))

movers foreach {v => v match {
    case h: helicopter => println(s"${h.color} helicopter")
    case s: submarine => println(s"${s.color} submarine")
}}

movers foreach { _ match {
    case helicopter("blue", owner) => println(s"$owner has a blue helicopter")
    case s: submarine => println(s"${s.driver}'s ${s.color} submarine")
    case _ =>
}}

## More Graceful Interactions with `Option`

* Many Scala operations pass over None

In [ ]:
val l = Seq.tabulate(5)(i => if (i % 2 == 1) Some(i) else None)
l.head foreach println
l flatten

l(1) match {
    case Some(i) => println(i)
    case None => println("was empty")
}